In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split

Add appts, [last month/year?], home sales[can't find current], and more stocks?

In [2]:
data = pd.read_csv("data/out/features.csv")
len(data), len(data.columns)

(59, 89)

In [3]:
X, y = data.drop(columns = ['n']), data['n']

In [4]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0, random_state=69)

In [5]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Lasso())
])

In [6]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.0, 30, 0.5)},
                      cv = 5,
                      scoring = 'neg_mean_squared_error',
                      verbose = 3)

In [7]:
search.fit(X, y)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END ............model__alpha=0.0;, score=-275582.484 total time=   0.0s
[CV 2/5] END .............model__alpha=0.0;, score=-36346.825 total time=   0.0s
[CV 3/5] END .............model__alpha=0.0;, score=-13052.674 total time=   0.0s
[CV 4/5] END .............model__alpha=0.0;, score=-19437.358 total time=   0.0s
[CV 5/5] END ............model__alpha=0.0;, score=-123954.609 total time=   0.0s
[CV 1/5] END .............model__alpha=0.5;, score=-33956.968 total time=   0.0s
[CV 2/5] END .............model__alpha=0.5;, score=-13366.182 total time=   0.0s
[CV 3/5] END ..............model__alpha=0.5;, score=-5951.394 total time=   0.0s
[CV 4/5] END ..............model__alpha=0.5;, score=-6607.073 total time=   0.0s
[CV 5/5] END .............model__alpha=0.5;, score=-69995.102 total time=   0.0s
[CV 1/5] END .............model__alpha=1.0;, score=-41903.119 total time=   0.0s
[CV 2/5] END ..............model__alpha=1.0;, s

C:\Users\keato\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py:394: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\keato\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
C:\Users\keato\AppData\Roaming\Python\Python37\site-packages\sklearn\pipeline.py:394: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\keato\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpecte

[CV 2/5] END ..............model__alpha=1.5;, score=-8807.140 total time=   0.0s
[CV 3/5] END ..............model__alpha=1.5;, score=-3785.261 total time=   0.0s
[CV 4/5] END ..............model__alpha=1.5;, score=-7905.334 total time=   0.0s
[CV 5/5] END .............model__alpha=1.5;, score=-54170.387 total time=   0.0s
[CV 1/5] END .............model__alpha=2.0;, score=-43922.523 total time=   0.0s
[CV 2/5] END ..............model__alpha=2.0;, score=-7190.306 total time=   0.0s
[CV 3/5] END ..............model__alpha=2.0;, score=-2707.275 total time=   0.0s
[CV 4/5] END ..............model__alpha=2.0;, score=-6587.147 total time=   0.0s
[CV 5/5] END .............model__alpha=2.0;, score=-43706.230 total time=   0.0s
[CV 1/5] END .............model__alpha=2.5;, score=-35594.594 total time=   0.0s
[CV 2/5] END ..............model__alpha=2.5;, score=-6218.788 total time=   0.0s
[CV 3/5] END ..............model__alpha=2.5;, score=-2121.396 total time=   0.0s
[CV 4/5] END ..............m

[CV 4/5] END .............model__alpha=12.5;, score=-1986.153 total time=   0.0s
[CV 5/5] END ............model__alpha=12.5;, score=-21142.350 total time=   0.0s
[CV 1/5] END .............model__alpha=13.0;, score=-1562.385 total time=   0.0s
[CV 2/5] END .............model__alpha=13.0;, score=-3892.498 total time=   0.0s
[CV 3/5] END .............model__alpha=13.0;, score=-1623.355 total time=   0.0s
[CV 4/5] END .............model__alpha=13.0;, score=-1956.567 total time=   0.0s
[CV 5/5] END ............model__alpha=13.0;, score=-21277.187 total time=   0.0s
[CV 1/5] END .............model__alpha=13.5;, score=-1470.543 total time=   0.0s
[CV 2/5] END .............model__alpha=13.5;, score=-3902.447 total time=   0.0s
[CV 3/5] END .............model__alpha=13.5;, score=-1631.025 total time=   0.0s
[CV 4/5] END .............model__alpha=13.5;, score=-1928.201 total time=   0.0s
[CV 5/5] END ............model__alpha=13.5;, score=-21394.293 total time=   0.0s
[CV 1/5] END .............mo

[CV 4/5] END .............model__alpha=23.0;, score=-1705.494 total time=   0.0s
[CV 5/5] END ............model__alpha=23.0;, score=-21605.664 total time=   0.0s
[CV 1/5] END .............model__alpha=23.5;, score=-1086.991 total time=   0.0s
[CV 2/5] END .............model__alpha=23.5;, score=-4417.261 total time=   0.0s
[CV 3/5] END .............model__alpha=23.5;, score=-1516.569 total time=   0.0s
[CV 4/5] END .............model__alpha=23.5;, score=-1698.573 total time=   0.0s
[CV 5/5] END ............model__alpha=23.5;, score=-21605.664 total time=   0.0s
[CV 1/5] END .............model__alpha=24.0;, score=-1081.426 total time=   0.0s
[CV 2/5] END .............model__alpha=24.0;, score=-4473.778 total time=   0.0s
[CV 3/5] END .............model__alpha=24.0;, score=-1506.958 total time=   0.0s
[CV 4/5] END .............model__alpha=24.0;, score=-1691.950 total time=   0.0s
[CV 5/5] END ............model__alpha=24.0;, score=-21605.664 total time=   0.0s
[CV 1/5] END .............mo

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,
        5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. , 10.5,
       11. , 11.5, 12. , 12.5, 13. , 13.5, 14. , 14.5, 15. , 15.5, 16. ,
       16.5, 17. , 17.5, 18. , 18.5, 19. , 19.5, 20. , 20.5, 21. , 21.5,
       22. , 22.5, 23. , 23.5, 24. , 24.5, 25. , 25.5, 26. , 26.5, 27. ,
       27.5, 28. , 28.5, 29. , 29.5])},
             scoring='neg_mean_squared_error', verbose=3)

In [8]:
search.best_params_

{'model__alpha': 7.0}

In [9]:
coef = search.best_estimator_[1].coef_

In [10]:
out = X.iloc[:, coef != 0]
out.columns

Index(['sentiment_lag6', 'new_units_lag12', 'GM_lag12', 'GM_v_lag12',
       'F_v_lag12', 'TSLA_v_lag3', 'AN_v_lag3', 'XOM_lag6',
       'g_best_new_cars_lag3', 'g_best_new_cars_lag12', 'oil_lag12'],
      dtype='object')

In [11]:
out.insert(0, 'n', y, True)
out.insert(0, 'month', X['month'], True)

In [12]:
out.to_csv("data/out/lasso.csv")

In [13]:
out

,month,n,sentiment_lag6,new_units_lag12,GM_lag12,GM_v_lag12,F_v_lag12,TSLA_v_lag3,AN_v_lag3,XOM_lag6,g_best_new_cars_lag3,g_best_new_cars_lag12,oil_lag12
0,1,352,93.1,3967.330793,32.62,322791897,596534720,100854710,19192133,79.21,68,85,44.83
1,2,330,91.9,4624.848400,37.31,325763066,524657536,78487987,15208883,75.24,71,76,56.83
2,3,323,87.2,3249.005847,37.50,290497517,609892208,59852189,17266632,74.35,76,77,51.06
3,4,369,90.0,2951.720587,35.06,397017858,539661690,79247152,39619066,82.74,77,87,62.21
4,5,392,91.3,3100.106707,35.97,247974427,428638676,133705697,45371013,81.66,74,79,60.47
5,6,329,92.6,3336.716121,33.33,301968673,577413238,102729544,33490119,77.95,74,86,58.79
6,7,420,92.0,3023.300368,31.51,421897537,672367234,135507264,40381923,77.85,76,100,50.51
7,8,359,91.7,3604.501487,29.44,388156191,596411983,103313344,23858141,80.15,74,96,47.01
8,9,368,91.0,3352.754923,30.02,356102158,743132834,121687759,26540639,83.59,71,76,43.58
9,10,367,89.0,2896.245248,34.91,324318496,669014321,74068654,25137021,88.40,75,68,43.66
